<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/weekly_CovidCases_CA_NY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic: Investigate Covid-19 NY and CA data

This colab includes CA and NY covid case data from JHU and moving avg trend analysis

In [36]:
!pip install wget

## Import Libraries

In [37]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

## Mount Google drive


In [38]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [39]:
from google.colab import drive
# drive.mount('/content/drive')

In [40]:
location = "drive/Shared drives/the-mean-sqaures/the-mean-squares/Colab_Dataset/Dataset/"

In [41]:
!ls /content/drive/My\ Drive/MasterProject-Personal/data

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data': No such file or directory


# Data Load

## 1 Load County Population


In [42]:
county_population_US = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)

(3195, 4)


In [43]:
wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv('covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)




(3195, 4)


In [44]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


## 2 Load Covid-19 case details (until July 12)

In [45]:
!ls '/content/drive/My Drive/MasterProject-Personal/data/'

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data/': No such file or directory


### Note about data:
John Hopkins university updates data every day hence we are pulling from repository directly

**US Confirmed url** :https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

**US deaths url**: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv

In [46]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

[wget.download(url) for url in urls]        

['time_series_covid19_confirmed_US (1).csv',
 'time_series_covid19_deaths_US (1).csv']

In [47]:
confirmed_US = pd.read_csv('time_series_covid19_confirmed_US.csv',low_memory=False)
death_US = pd.read_csv('time_series_covid19_deaths_US.csv',low_memory=False)
print(confirmed_US.shape)
print(death_US.shape)
print(confirmed_US.head(2))
death_US.head(2)

(3340, 223)
(3340, 224)
        UID iso2 iso3  code3  ...  8/17/20 8/18/20 8/19/20 8/20/20
0  84001001   US  USA    840  ...     1219    1235    1241    1240
1  84001003   US  USA    840  ...     3888    3906    3931    3957

[2 rows x 223 columns]


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15,15,17,18,19,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,21,21,21,21,21,21,22,22,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,11,11,11,12,13,13,14,14,14,15,15,16,17,17,17,17,17,20,20,21,21,22,23,23,23,23,23,24,25,25,29,29,29,29,29,29,29,30,30,31


In [48]:
#print(len(mask_data['state_name'].unique()))
print(len(confirmed_US['Province_State'].unique()))
confirmed_US['Province_State'].unique()

58


array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

## Data cleaning

In [49]:
confirmed_US.columns[:11]

# Note: the first 11 columns contains UID, ios2,ios3, code ,FIPS, 'Admin2', 'Province_State',
      # 'Country_Region', 'Lat', 'Long_',Combined_Key', 'Population',


Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],
      dtype='object')

In [50]:
# date begins from 11th column
confirmed_dates = confirmed_US.columns[11:]
confirmed_dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '8/11/20', '8/12/20', '8/13/20', '8/14/20', '8/15/20', '8/16/20',
       '8/17/20', '8/18/20', '8/19/20', '8/20/20'],
      dtype='object', length=212)

In [51]:
death_US.columns[:12]

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
      dtype='object')

In [52]:
death_US.columns[10:]

Index(['Combined_Key', 'Population', '1/22/20', '1/23/20', '1/24/20',
       '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       ...
       '8/11/20', '8/12/20', '8/13/20', '8/14/20', '8/15/20', '8/16/20',
       '8/17/20', '8/18/20', '8/19/20', '8/20/20'],
      dtype='object', length=214)

In [53]:
death_dates = death_US.columns[12:]
death_dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '8/11/20', '8/12/20', '8/13/20', '8/14/20', '8/15/20', '8/16/20',
       '8/17/20', '8/18/20', '8/19/20', '8/20/20'],
      dtype='object', length=212)

#### Note: both the date values has the same beginning date 1/22/2020. Hence we can use either of the values

In [54]:
confirmed_df_long = confirmed_US.melt(
    id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],
       value_vars=confirmed_dates,
       var_name = 'Date',
       value_name = 'Confirmed'
)

death_df_long = death_US.melt(
    id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
       value_vars=death_dates,
       var_name = 'Date',
       value_name = 'Deaths'
)

In [55]:
confirmed_df_long.tail(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
708070,84056029,US,USA,840,56029.0,Park,Wyoming,US,44.521575,-109.585282,"Park, Wyoming, US",8/20/20,153
708071,84056031,US,USA,840,56031.0,Platte,Wyoming,US,42.132991,-104.966331,"Platte, Wyoming, US",8/20/20,7
708072,84056033,US,USA,840,56033.0,Sheridan,Wyoming,US,44.790489,-106.886239,"Sheridan, Wyoming, US",8/20/20,108
708073,84056035,US,USA,840,56035.0,Sublette,Wyoming,US,42.765583,-109.913092,"Sublette, Wyoming, US",8/20/20,41
708074,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",8/20/20,285
708075,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",8/20/20,398
708076,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",8/20/20,283
708077,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",8/20/20,0
708078,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",8/20/20,106
708079,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",8/20/20,0


In [56]:
death_df_long.tail(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,Date,Deaths
708070,84056029,US,USA,840,56029.0,Park,Wyoming,US,44.521575,-109.585282,"Park, Wyoming, US",29194,8/20/20,0
708071,84056031,US,USA,840,56031.0,Platte,Wyoming,US,42.132991,-104.966331,"Platte, Wyoming, US",8393,8/20/20,0
708072,84056033,US,USA,840,56033.0,Sheridan,Wyoming,US,44.790489,-106.886239,"Sheridan, Wyoming, US",30485,8/20/20,0
708073,84056035,US,USA,840,56035.0,Sublette,Wyoming,US,42.765583,-109.913092,"Sublette, Wyoming, US",9831,8/20/20,0
708074,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",42343,8/20/20,0
708075,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464,8/20/20,0
708076,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226,8/20/20,0
708077,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,8/20/20,33
708078,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805,8/20/20,0
708079,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",6927,8/20/20,0


## Check California-New York data

In [57]:
state= ['California','New York']
confirmed_CA_df = confirmed_df_long[confirmed_df_long['Province_State'].isin(state)]
confirmed_CA_df.tail(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
706699,84036115,US,USA,840,36115.0,Washington,New York,US,43.311538,-73.430434,"Washington, New York, US",8/20/20,264
706700,84036117,US,USA,840,36117.0,Wayne,New York,US,43.154944,-77.029765,"Wayne, New York, US",8/20/20,270
706701,84036119,US,USA,840,36119.0,Westchester,New York,US,41.162784,-73.757417,"Westchester, New York, US",8/20/20,36586
706702,84036121,US,USA,840,36121.0,Wyoming,New York,US,42.701451,-78.221996,"Wyoming, New York, US",8/20/20,120
706703,84036123,US,USA,840,36123.0,Yates,New York,US,42.635055,-77.103699,"Yates, New York, US",8/20/20,59


In [58]:
confirmed_CA_df.Province_State.unique()

array(['California', 'New York'], dtype=object)

## Merging Confirmed and Death data

In [59]:
full_table = confirmed_df_long.merge(
    right=death_df_long,
    how='left',
    on=[ 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key','Date']
)

full_table.head(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Population,Deaths
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1/22/20,0,55869,0
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",1/22/20,0,223234,0
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",1/22/20,0,24686,0
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",1/22/20,0,22394,0
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1/22/20,0,57826,0
5,84001011,US,USA,840,1011.0,Bullock,Alabama,US,32.100305,-85.712655,"Bullock, Alabama, US",1/22/20,0,10101,0
6,84001013,US,USA,840,1013.0,Butler,Alabama,US,31.753001,-86.680575,"Butler, Alabama, US",1/22/20,0,19448,0
7,84001015,US,USA,840,1015.0,Calhoun,Alabama,US,33.774837,-85.826304,"Calhoun, Alabama, US",1/22/20,0,113605,0
8,84001017,US,USA,840,1017.0,Chambers,Alabama,US,32.913601,-85.390727,"Chambers, Alabama, US",1/22/20,0,33254,0
9,84001019,US,USA,840,1019.0,Cherokee,Alabama,US,34.178060,-85.606390,"Cherokee, Alabama, US",1/22/20,0,26196,0


In [60]:
#full_table['Date'] = pd.to_datetime(full_table['Date'])


In [61]:
ship_data = full_table['Province_State'].str.contains('Grand Princess') | full_table['Province_State'].str.contains('Diamond Princess') | full_table['Province_State'].str.contains('Northern Mariana Islands') | full_table['Province_State'].str.contains('American Samoa') |full_table['Province_State'].str.contains('Guam') | full_table['Province_State'].str.contains('Virgin Islands')

full_ship = full_table[ship_data]


In [62]:
# Removing ship data from State data

full_table = full_table[~(ship_data)]

## Group data

In [97]:
full_grouped = full_table.groupby(['Date', 'Province_State','FIPS'])['Confirmed', 'Deaths'].sum().reset_index()

full_grouped.tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths
704683,8/9/20,Wyoming,56041.0,278,0
704684,8/9/20,Wyoming,56043.0,77,0
704685,8/9/20,Wyoming,56045.0,5,0
704686,8/9/20,Wyoming,80056.0,0,0
704687,8/9/20,Wyoming,90056.0,0,27


In [64]:
full_grouped.shape

(704688, 5)

####  Adding new cases, new deaths by subtracting from previous day record

In [65]:
full_grouped.dtypes

Date               object
Province_State     object
FIPS              float64
Confirmed           int64
Deaths              int64
dtype: object

In [98]:
full_grouped_ = full_grouped.copy()

In [124]:
full_grouped = full_grouped_.copy()

In [151]:
df = pd.DataFrame(columns=['Date','Province_State','FIPS','Confirmed','Deaths','New cases','New deaths'])



In [152]:
def fixDate(x):
  arr = x.split('/')
  m = arr[0]
  d = arr[1]
  y = arr[2]

  if int(m) < 10:
    m = '0'+str(m)
  if int(d) < 10:
    d = '0'+str(d)
  return '20'+str(y)+'-'+m+'-'+d

In [148]:
full_grouped['Date'] = full_grouped['Date'].apply(lambda x: fixDate(x))

full_grouped.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths
0,2020-01-22,Alabama,1001.0,0,0
1,2020-01-22,Alabama,1003.0,0,0


In [149]:
def computeNewCases(FIPS):
  countyData = full_grouped[full_grouped['FIPS'] == FIPS]
  countyData = countyData.sort_values(by=['Date'])
  temp = countyData.groupby(['Date'])['Confirmed', 'Deaths']
  temp = temp.sum().diff().reset_index()
  #print(temp)
  temp.loc[mask, 'Confirmed'] = np.nan
  temp.loc[mask, 'Deaths'] = np.nan
  temp.columns = ['Date', 'New cases', 'New deaths']
  countyData = pd.merge(countyData, temp, on=[ 'Date'])
  #print(countyData)
  #print('********')
  # filling na with 0
  countyData = countyData.fillna(0)
  # fixing data types
  cols = ['New cases', 'New deaths']
  countyData[cols] = countyData[cols].astype('int') 
  return countyData


Below code executes for all FIPS, estimated runtime **10 minutes**

In [153]:

for fips in full_grouped.FIPS.unique():
  countydf = computeNewCases(fips)
  df = df.append(countydf)

In [158]:
df.head(30)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,Alabama,1001.0,0,0,0,0
1,2020-01-23,Alabama,1001.0,0,0,0,0
2,2020-01-24,Alabama,1001.0,0,0,0,0
3,2020-01-25,Alabama,1001.0,0,0,0,0
4,2020-01-26,Alabama,1001.0,0,0,0,0
5,2020-01-27,Alabama,1001.0,0,0,0,0
6,2020-01-28,Alabama,1001.0,0,0,0,0
7,2020-01-29,Alabama,1001.0,0,0,0,0
8,2020-01-30,Alabama,1001.0,0,0,0,0
9,2020-01-31,Alabama,1001.0,0,0,0,0


In [160]:
df[df['FIPS'] == 56041.0].tail(50)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
162,2020-07-02,Wyoming,56041.0,180,0,3,0
163,2020-07-03,Wyoming,56041.0,182,0,2,0
164,2020-07-04,Wyoming,56041.0,183,0,1,0
165,2020-07-05,Wyoming,56041.0,184,0,1,0
166,2020-07-06,Wyoming,56041.0,190,0,6,0
167,2020-07-07,Wyoming,56041.0,190,0,0,0
168,2020-07-08,Wyoming,56041.0,192,0,2,0
169,2020-07-09,Wyoming,56041.0,198,0,6,0
170,2020-07-10,Wyoming,56041.0,200,0,2,0
171,2020-07-11,Wyoming,56041.0,201,0,1,0


In [161]:
# merging new values
full_grouped = df.copy()

In [162]:
full_grouped.tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
207,2020-08-16,Wyoming,90056.0,0,29,0,0
208,2020-08-17,Wyoming,90056.0,0,29,0,0
209,2020-08-18,Wyoming,90056.0,0,29,0,0
210,2020-08-19,Wyoming,90056.0,0,33,0,4
211,2020-08-20,Wyoming,90056.0,0,33,0,0


In [163]:
state= ['California','New York']
ca_df = full_grouped[full_grouped['Province_State'].isin(state)]
ca_df.tail(10)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
202,2020-08-11,New York,90036.0,0,0,0,0
203,2020-08-12,New York,90036.0,0,0,0,0
204,2020-08-13,New York,90036.0,0,0,0,0
205,2020-08-14,New York,90036.0,0,0,0,0
206,2020-08-15,New York,90036.0,0,0,0,0
207,2020-08-16,New York,90036.0,0,0,0,0
208,2020-08-17,New York,90036.0,0,0,0,0
209,2020-08-18,New York,90036.0,0,0,0,0
210,2020-08-19,New York,90036.0,0,0,0,0
211,2020-08-20,New York,90036.0,0,0,0,0


In [69]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


In [70]:
county_population_US['FIPS'] = county_population_US['countyFIPS']

In [71]:
ca_df.shape

(26288, 7)

# Merge with FIPS 

In [72]:
merged = pd.merge(ca_df,county_population_US,how='inner' ,on=['FIPS'])
print(merged.shape)

(25440, 11)


In [73]:
merged.tail(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,countyFIPS,County Name,State,population
25438,8/8/20,New York,36123.0,56,7,0,2,36123,Yates County,NY,24913
25439,8/9/20,New York,36123.0,57,7,0,2,36123,Yates County,NY,24913


## Visualization

In [74]:
full_grouped.head(3)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,1/22/20,Alabama,1001.0,0,0,0,0
1,1/22/20,Alabama,1003.0,0,0,0,0
2,1/22/20,Alabama,1005.0,0,0,0,0


In [75]:
merged.head(3)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,countyFIPS,County Name,State,population
0,1/22/20,California,6001.0,0,0,0,0,6001,Alameda County,CA,1671329
1,1/23/20,California,6001.0,0,0,0,0,6001,Alameda County,CA,1671329
2,1/24/20,California,6001.0,0,0,0,0,6001,Alameda County,CA,1671329


In [76]:
import pandas as pd
import altair as alt
#full_grouped = merged
ca_df = full_grouped[full_grouped['Province_State'] == 'California']
ny_df = full_grouped[full_grouped['Province_State'] == 'New York']
queens_df = full_grouped[(full_grouped['Province_State'] == 'New York') & (full_grouped['FIPS'] == 36081)]

ca_df.shape

(12720, 7)

In [77]:
ca_total = (
    ca_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)
ny_total = (
    ny_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)

queens_county_total = (
    queens_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)

In [78]:
base_ca = alt.Chart(ca_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_ny = alt.Chart(ny_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_queens = alt.Chart(queens_county_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

In [79]:
red = alt.value("#f54242")
##Ca data
#base_ca.encode(y='Confirmed').properties(title='Total Confirmed') | base_ca.encode(y='Deaths',color = red).properties(title='Total deaths')
base_ca.encode(y='New cases').properties(title='CA State- New cases')

alt.Chart(...)

In [80]:
#base_ny.encode(y='Confirmed').properties(title='Total Confirmed') | base_ny.encode(y='Deaths',color = red).properties(title='Total deaths')

base_ny.encode(y='New cases').properties(title='NY state - New cases')


alt.Chart(...)

In [81]:
base_queens.encode(y="New cases").properties(title='Queens County New Cases')

alt.Chart(...)

# Rolling Average

In [82]:
queens_county_total.head()

,Date,Province_State,New cases
0,1/22/20,New York,0
1,1/23/20,New York,0
2,1/24/20,New York,0
3,1/25/20,New York,0
4,1/26/20,New York,0


In [83]:
queens_county_total.iloc[:,2]

0      0
1      0
2      0
3      0
4      0
      ..
207    0
208    0
209    0
210    0
211    0
Name: New cases, Length: 212, dtype: int64

In [84]:
base_queens.encode(y="New cases").properties(title='Queens County New Cases')

alt.Chart(...)

In [85]:
queens_county_total['rolling_average'] = queens_county_total.iloc[:,2].rolling(window=7).mean()

base_queens = alt.Chart(queens_county_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)


In [86]:
queens_county_total.tail(10)

,Date,Province_State,New cases,rolling_average
202,8/19/20,New York,0,0.0
203,8/2/20,New York,0,0.0
204,8/20/20,New York,0,0.0
205,8/3/20,New York,0,0.0
206,8/4/20,New York,0,0.0
207,8/5/20,New York,0,0.0
208,8/6/20,New York,0,0.0
209,8/7/20,New York,0,0.0
210,8/8/20,New York,0,0.0
211,8/9/20,New York,0,0.0


In [87]:
base_queens.encode(y="rolling_average").properties(title='Queens County- 7 day rolling average (New Cases)')


alt.Chart(...)

In [88]:
bar = base_queens.mark_bar().encode(y="New cases")

line =  base_queens.mark_line(color='red').encode(
    y='rolling_average'
)

(bar + line).properties(title='Queens County - New Cases and rolling average ',width=600)

alt.LayerChart(...)

# New Section

In [89]:
fractalData_combo1

NameError: ignored

In [ ]:
test = fractalData_combo1[fractalData_combo1['County Name'] == 'San Diego County']
test.shape
test

In [ ]:
cluster_perf_df.head()

In [ ]:
cluster_perf_df1.head() # min

In [ ]:
cluster_perf_df1_2.head() #max

In [ ]:
test1 = pd.merge(test,week_df, how='inner', on ='WeekNumber')
test1.tail(10)

In [ ]:
red = alt.value("#f54242")
base_orange.encode(y='Confirmed').properties(title='Total Confirmed') 

# Result Comparison

In [ ]:
# considering for Final comparison
combo1_cluster2

In [ ]:
combo2_cluster1

In [ ]:
combo2_cluster2

In [ ]:
combo3_cluster1

In [ ]:
combo3_cluster2

## Data Exploration

In [ ]:
import altair as alt

In [ ]:
week_df

In [ ]:
merged.dtypes

In [ ]:
LA_county = merged[merged['County Name'] == 'Los Angeles County']
LA_county.shape

In [ ]:
orange_county = merged[merged['County Name'] == 'San Diego County']


In [ ]:
base_la = alt.Chart(LA_county).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_orange = alt.Chart(orange_county).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

In [ ]:
red = alt.value("#f54242")
base_orange.encode(y='Confirmed').properties(title='Total Confirmed') 


In [164]:
ca_df = full_grouped[full_grouped['Province_State'] == 'California']
ny_df = full_grouped[full_grouped['Province_State'] == 'New York']
tx_df = full_grouped[full_grouped['Province_State'] == 'Texas']

In [166]:
ny_df.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,New York,36001.0,0,0,0,0
1,2020-01-23,New York,36001.0,0,0,0,0
2,2020-01-24,New York,36001.0,0,0,0,0
3,2020-01-25,New York,36001.0,0,0,0,0
4,2020-01-26,New York,36001.0,0,0,0,0


In [167]:
ny_df.to_csv('NY-CovidAug21.csv',index=False)

In [ ]:
base_ca = alt.Chart(ca_df).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_ny = alt.Chart(ny_df).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_tx = alt.Chart(tx_df).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

In [ ]:
red = alt.value("#f54242")
base_ca.encode(y='Confirmed').properties(title='Total Confirmed') | base_ca.encode(y='Deaths',color = red).properties(title='Total deaths')


In [ ]:
base_ny.encode(y='Confirmed').properties(title='Total Confirmed') | base_ny.encode(y='Deaths',color = red).properties(title='Total deaths')


In [ ]:
base_tx.encode(y='Confirmed').properties(title='Total Confirmed') | base_tx.encode(y='Deaths',color = red).properties(title='Total deaths')


In [ ]:
base_ca.encode(y='New cases').properties(title='Total New Cases') | base_ca.encode(y='New deaths',color = red).properties(title='Total New deaths')


## Steps to approach the problem

* Predict how 'New cases' count changed in states after mask mandate
* check the type of **rule** - 
  1. Entire State
  2. Entire Territory
  3. Parts of State
  4. Entire State (Employees Only)
  5. Parts of State (Employees Only)
  6. No
  7. Masks strongly recommended, provides masks for free
  8. Entire State (Some Employees)

impacts the new cases count

* Include/identify latent variables like 'Long weekend' date, 'Rally' date, 'BLM' protests happened date impact in new cases count 
* Identify datasource for people cooperation in following the rule
* Political party ruling the state
* Population 
* Epicenter city of each state
* Type of mask (?)



## Reference

* https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6